In [3]:
from sklearn.utils.extmath import softmax
import numpy as np

In [5]:
x = np.array([[0.5, 0]])
softmax(x)

array([[0.62245933, 0.37754067]])

In [6]:
x = np.array([[0.5, 0.5]])
softmax(x)

array([[0.5, 0.5]])

In [11]:
x = np.array([[0, 0.99]])
softmax(x)

array([[0.27091208, 0.72908792]])

In [14]:
import networkx as nx
import pickle
import json

In [15]:
G = nx.read_gpickle("../data/graph_description_update.pickle")

In [16]:
len(G)

2583

In [17]:
G['active']

AtlasView({'passive': {'weight': -1}, 'involved': {'weight': 1}, 'inactive': {'weight': -1}, 'an': {'relation': 'defined as'}, 'asserting that the person or thing represented by the grammatical subject performs the action represented by the verb': {'relation': 'defined as'}, 'capable of': {'relation': 'defined as'}, 'capable of controlling': {'relation': 'defined as'}, 'characterized by': {'relation': 'defined as'}, 'characterized by emission of large amounts of electromagnetic energy': {'relation': 'defined as'}, 'characterized or accomplished by action or effort': {'relation': 'defined as'}, 'currently erupting or likely to erupt': {'relation': 'defined as'}, 'disposed to action': {'relation': 'defined as'}, 'doing things that require physical movement and energy': {'relation': 'defined as'}, 'engaged in an action or activity': {'relation': 'defined as'}, 'engaged in full-time service especially in the armed forces': {'relation': 'defined as'}, 'engaged or participating in action or 

In [19]:
for node in list(G):
    if node.find(' ') > -1:
        G.remove_node(node)
len(G)

757

In [21]:
G.nodes()

NodeView(('active', 'considerate', 'convivial', 'diplomatic', 'gregarious', 'optimistic', 'mature', 'motivated', 'sensible', 'brave', 'chatty', 'clever', 'smart', 'cowardly', 'easy-going', 'friendly', 'funny', 'generous', 'grumpy', 'honest', 'kind', 'lazy', 'loud', 'lucky', 'moody', 'nasty', 'neat', 'nervous', 'nice', 'polite', 'popular', 'quiet', 'rude', 'selfish', 'serious', 'shy', 'silly', 'stupid', 'tidy', 'unlucky', 'untidy', 'vain', 'wise', 'adaptable', 'adventurous', 'affectionate', 'ambitious', 'amiable', 'compassionate', 'courageous', 'courteous', 'diligent', 'empathetic', 'exuberant', 'frank', 'impartial', 'intuitive', 'inventive', 'passionate', 'persistent', 'philosophical', 'practical', 'rational', 'reliable', 'resourceful', 'sincere', 'sympathetic', 'unassuming', 'witty', 'argumentative', 'cynical', 'disruptive', 'inconsiderate', 'passive', 'resentful', 'curious', 'excellent', 'responsible', 'affable', 'eager', 'outstanding', 'agreeable', 'encouraging', 'positive', 'enthus

In [49]:
import pandas as pd

In [50]:
df = pd.DataFrame(columns=['C', 'E', 'N'])
df.head()

,C,E,N


In [51]:
import random

def generate_triplet():
    C = random.random()
    E = random.random()*(1-C)
    N = 1 - E - C
    return C, E, N

In [52]:
for i in range(0,10):
    C, E, N = generate_triplet()
    row = pd.DataFrame([[C, E, N]], columns=['C', 'E', 'N'])
    df = df.append(row, ignore_index=True)
df

,C,E,N
0,0.388676,0.149681,0.461643
1,0.920901,0.006956,0.072143
2,0.258320,0.422925,0.318755
3,0.730319,0.014069,0.255611
4,0.344979,0.108929,0.546092
5,0.242382,0.698794,0.058824
6,0.201700,0.197483,0.600817
7,0.842285,0.042234,0.115481
8,0.110559,0.538799,0.350642
9,0.314546,0.367038,0.318416


In [54]:
def gen_dist(row):
    C = row['C']
    E = row['E']
    N = row['N']
    
    if C > 0.5:
        return C
    elif C == E:
        return 0.5
    else:
        return N/(N+E)*0.5

In [66]:
ser_res = df.apply(lambda row: gen_dist(row), axis=1)
ser_res.name = 'dist'

In [67]:
df = df.join(ser_res)
df.head()

,C,E,N,dist
0,0.388676,0.149681,0.461643,0.377577
1,0.920901,0.006956,0.072143,0.920901
2,0.258320,0.422925,0.318755,0.214887
3,0.730319,0.014069,0.255611,0.730319
4,0.344979,0.108929,0.546092,0.416851


In [68]:
# looks ok

In [80]:
def non_neutral_score(row):
    """
    dist is a float value between (0,1), where 0 represents total entailment,
    0.5 represents total independence (neutrality), and 1 represents total contradiction.
    """
    dist = row['dist']
    score = 4*(dist-0.5)*(dist-0.5)
    return score

In [81]:
ser_non_neut = df.apply(lambda row: non_neutral_score(row), axis=1)
ser_non_neut.name = 'non_neutrality'

df = df.join(ser_non_neut)
df

In [84]:
from math import sqrt
ser_non_neut_sqrt = df.apply(lambda row: sqrt(row['non_neutrality']), axis=1)
ser_non_neut_sqrt.name = 'sqrt_non_neutrality'

df = df.join(ser_non_neut_sqrt)
df

,C,E,N,dist,non_neutrality,sqrt_non_neutrality
0,0.388676,0.149681,0.461643,0.377577,0.059950,0.244847
1,0.920901,0.006956,0.072143,0.920901,0.708632,0.841803
2,0.258320,0.422925,0.318755,0.214887,0.325158,0.570226
3,0.730319,0.014069,0.255611,0.730319,0.212188,0.460639
4,0.344979,0.108929,0.546092,0.416851,0.027655,0.166298
5,0.242382,0.698794,0.058824,0.038822,0.850741,0.922356
6,0.201700,0.197483,0.600817,0.376310,0.061196,0.247379
7,0.842285,0.042234,0.115481,0.842285,0.468635,0.684569
8,0.110559,0.538799,0.350642,0.197114,0.366960,0.605772
9,0.314546,0.367038,0.318416,0.232266,0.286725,0.535467
